In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv('valid.csv')

In [ ]:
with open("C:\\Users\\vidus\\Desktop\\ml hw\\a.txt", 'r') as file:
    for line in file:
        col = line.strip()
        data = data.drop(col, axis=1)

for col in data.columns:
    if data[col].isnull().sum() > 103558:
        data = data.drop(col, axis=1)

missing_values_count = data.isnull().sum()

plt.figure(figsize=(10, 50))
missing_values_count.plot(kind='barh', color='skyblue')
plt.title('Missing Values in Columns')
plt.xlabel('Columns')
plt.ylabel('Number of Missing Values')
plt.xticks(rotation=45, ha='right')
plt.show()


In [ ]:
cols = ["home_ownership", "verification_status", "term", "grade", "initial_list_status", "debt_settlement_flag", "emp_length", "initial_list_status"
]   
data_encoded = pd.get_dummies(data, columns=cols)


y = data_encoded["loan_status"]

data_encoded = data_encoded.drop("loan_status", axis=1)

cols1 = ["purpose", "title", "emp_title", "issue_d", "zip_code", "addr_state", "earliest_cr_line", "sub_grade", "last_pymnt_d", "last_credit_pull_d"]
for col in cols1:
    dr_frequency_map=data_encoded[col].value_counts().to_dict()
    data_encoded[col] = data_encoded[col].map(dr_frequency_map)

In [ ]:
from scipy.stats import skew

# Identify skewed columns (numeric)
numeric_cols = data_encoded.select_dtypes(include=['float64', 'int64']).columns
skewed_cols = data_encoded[numeric_cols].apply(lambda x: skew(x.dropna()))

# Choose a skewness threshold (you can adjust this value)
skew_threshold = 0.5
skewed_cols = skewed_cols[abs(skewed_cols) > skew_threshold].index

# Replace missing values with the median in skewed columns
for col in skewed_cols:
    median_value = data_encoded[col].median()
    data_encoded[col].fillna(median_value, inplace=True)

# Verify that missing values are replaced
# data.isnull().sum()  # Uncomment to check for missing values after replacement
    
fill_mean = ["bc_util", "revol_util", "mo_sin_old_il_acct", "percent_bc_gt_75"]

for col in fill_mean:
    data_encoded[col].fillna(data[col].mean(), inplace=True)


missing_values = data_encoded.isnull().sum()

data_encoded = data_encoded.fillna(value=0)

# Plot a bar graph
plt.figure(figsize=(10, 50))
missing_values.plot(kind='barh', color='skyblue')
plt.title('Missing Values in Columns')
plt.xlabel('Columns')
plt.ylabel('Number of Missing Values')
plt.xticks(rotation=45, ha='right')
plt.show()


In [ ]:
#week corr
col_to_drop = ["num_rev_tl_bal_gt_0", "issue_d", "earliest_cr_line", "last_pymnt_d", "last_credit_pull_d", "annual_inc", "dti", "delinq_2yrs", "inq_last_6mths", "total_il_high_credit_limit"]
data_encoded = data_encoded.drop(col_to_drop, axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
scaler.fit(data_encoded)
scaled_data=scaler.transform(data_encoded)

from sklearn.decomposition import PCA

pca=PCA(n_components=20,random_state=42)
pca.fit(scaled_data)
X_pca=pca.transform(scaled_data)

from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# fit model no training data
model = XGBClassifier()
model.fit(X_pca, y)

In [ ]:
# make predictions for test data
y_pred = model.predict(X_pca)
predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
final_result_df = pd.DataFrame(data_encoded)
final_result_df.insert(0, 'loan_status', y_pred)
final_result_df.to_csv('210099V.csv', index=False)